# Téléchargement des Données Historiques
Dans cette cellule, nous ajoutons le symbole BTCUSDT et téléchargeons les données historiques nécessaires pour préparer l'analyse.# Téléchargement des Données Historiques
Dans cette cellule, nous ajoutons le symbole BTCUSDT et téléchargeons les données historiques nécessaires pour préparer l'analyse.

In [ ]:
# Import des bibliothèques nécessaires
from AlgorithmImports import *
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import talib

qb = QuantBook()

# Ajouter BTCUSDT
symbol = qb.AddCrypto("BTCUSDT", Resolution.Daily, Market.Binance).Symbol
start_date, end_date = datetime(2017, 1, 1), datetime(2023, 10, 20)

# Récupération des données
history = qb.History([symbol], start_date, end_date, Resolution.Daily)
btc_prices = history.loc[symbol]

# Calcul des Indicateurs
Nous calculons les indicateurs nécessaires : les moyennes exponentielles (EMAs), l'ATR (volatilité historique), et le RSI (force relative). Ces indicateurs seront utilisés pour générer des signaux d'achat et de vente.

In [15]:
# Paramètres
fast_period, slow_period = 18, 50

# Calcul des EMA
btc_prices['FastEMA'] = btc_prices['close'].ewm(span=fast_period, adjust=False).mean()
btc_prices['SlowEMA'] = btc_prices['close'].ewm(span=slow_period, adjust=False).mean()

# ATR et RSI
btc_prices['TR'] = talib.TRANGE(btc_prices['high'], btc_prices['low'], btc_prices['close'])
btc_prices['ATR'] = talib.SMA(btc_prices['TR'], timeperiod=14)
btc_prices['RSI'] = talib.RSI(btc_prices['close'], timeperiod=14)

# Application des Filtres
Nous ajustons deux filtres :
1. Un filtre basé sur la volatilité (ATR) pour éviter les transactions dans des périodes financières instables.
2. Un filtre basé sur le RSI pour éviter d'acheter lorsque l'actif est suracheté.

In [16]:
# Filtre de volatilité
volatility_multiplier = 2.5
btc_prices['ValidTrade'] = btc_prices['ATR'] < btc_prices['ATR'].mean() * volatility_multiplier

# Filtre RSI
btc_prices['RSI_Filter'] = btc_prices['RSI'] < 50

# Génération des Signaux d'Achat et de Vente
Nous définissons les conditions pour générer des signaux d'achat et de vente. 
- Achat : croisement de FastEMA au-dessus de SlowEMA + filtres RSI et ATR.
- Vente : croisement de FastEMA en-dessous de SlowEMA + filtre ATR.

In [17]:
# Définition des signaux
up_cross_margin, down_cross_margin = 1.002, 0.998
btc_prices['BuySignal'] = (
    (btc_prices['FastEMA'] > btc_prices['SlowEMA'] * up_cross_margin) &
    btc_prices['ValidTrade'] & btc_prices['RSI_Filter']
)
btc_prices['SellSignal'] = (
    (btc_prices['FastEMA'] < btc_prices['SlowEMA'] * down_cross_margin) &
    btc_prices['ValidTrade']
)

# Sizing Dynamique
Nous calculons la taille de position dynamique. La taille des positions est inversément proportionnelle à la volatilité relative (ATR).

In [18]:
# Position dynamique - Correction dtype
btc_prices['PositionSize'] = (1 / (btc_prices['ATR'] / btc_prices['close'] + 1)).astype(float)

btc_prices['Position'] = 0
btc_prices.loc[btc_prices['BuySignal'], 'Position'] = btc_prices['PositionSize']
btc_prices.loc[btc_prices['SellSignal'], 'Position'] = -btc_prices['PositionSize']

# Décalage des positions
btc_prices['Position'] = btc_prices['Position'].shift(1).fillna(0)

# Création des Positions
Nous appliquons les signaux d'achat et de vente pour définir des positions (`1` pour l'achat, `-1` pour la vente, et `0` neutre). Les positions sont décalées pour éviter le lookahead bias.

In [19]:
btc_prices['Position'] = 0
btc_prices.loc[btc_prices['BuySignal'], 'Position'] = btc_prices['PositionSize']
btc_prices.loc[btc_prices['SellSignal'], 'Position'] = -btc_prices['PositionSize']

# Décalage pour éviter le lookahead bias
btc_prices['Position'] = btc_prices['Position'].shift(1)

# Calcul des Performances
Les rendements de la stratégie et la performance cumulée sont calculés ici. Nous comparons également les performances de la stratégie avec un investissement Buy-and-Hold.

In [20]:
btc_prices['DailyReturn'] = btc_prices['close'].pct_change()
btc_prices['StrategyReturn'] = btc_prices['Position'] * btc_prices['DailyReturn']
btc_prices['CumulativeMarket'] = (1 + btc_prices['DailyReturn']).cumprod()
btc_prices['CumulativeStrategy'] = (1 + btc_prices['StrategyReturn']).cumprod()

# Visualisation des Résultats
Nous traçons les performances cumulatives de la stratégie et les comparons au marché (Buy-and-Hold).

In [21]:
plt.figure(figsize=(12, 6))
plt.plot(btc_prices['CumulativeMarket'], label='Market Return (Buy & Hold)')
plt.plot(btc_prices['CumulativeStrategy'], label='Strategy Return')
plt.legend()
plt.title('Performance Cumulative de la Stratégie (Optimisée)')
plt.show()

# Analyse des Résultats
Nous calculons les métriques importantes de performance : Profit Total, Sharpe Ratio et Max Drawdown.
Nous examinons également les occurrences des signaux d'achat et de vente.

In [22]:
total_return = btc_prices['CumulativeStrategy'].iloc[-1] - 1
sharpe_ratio = btc_prices['StrategyReturn'].mean() / btc_prices['StrategyReturn'].std() * np.sqrt(252)
max_drawdown = (btc_prices['CumulativeStrategy'] / btc_prices['CumulativeStrategy'].cummax() - 1).min()

print(f"Total Return: {total_return:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Max Drawdown: {max_drawdown:.2%}")

# Compte des signaux
buy_signals = len(btc_prices.loc[btc_prices['BuySignal']])
sell_signals = len(btc_prices.loc[btc_prices['SellSignal']])
print(f"Nombre de signaux d'achat : {buy_signals}, Nombre de signaux de vente : {sell_signals}")